<a href="https://colab.research.google.com/github/alimomennasab/ChestXRay-Classification/blob/main/ResNext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notes: ResNext is combo of VGGNet, ResNet, and InceptionNet

##1. Setup

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import pandas as pd
import math
import os
import numpy as np
from PIL import Image
from glob import glob
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [46]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

##2. Loading Data

In [47]:
data_dir = '/content/drive/My Drive/chest_xray/'

In [48]:
# Split dataset into training, validation, and test sets
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

In [49]:
print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['train', 'chest_xray', 'val', '__MACOSX', 'test']
['PNEUMONIA', 'NORMAL']


In [50]:
# Pneumonia images
pneumonia_files = os.listdir(data_dir + "/train/PNEUMONIA")
print('No. of training examples for Pneumonia:', len(pneumonia_files))
print(pneumonia_files[:5])

No. of training examples for Pneumonia: 3875
['person556_virus_1096.jpeg', 'person536_bacteria_2257.jpeg', 'person581_bacteria_2390.jpeg', 'person592_bacteria_2434.jpeg', 'person581_virus_1125.jpeg']


In [51]:
# Normal (healthy) images
normal_files = os.listdir(data_dir + "/train/NORMAL")
print('No. of training examples for Normal:', len(normal_files))
print(normal_files[:5])

No. of training examples for Normal: 1341
['IM-0526-0001.jpeg', 'IM-0524-0001.jpeg', 'IM-0507-0001.jpeg', 'IM-0508-0001.jpeg', 'IM-0520-0001.jpeg']


In [52]:
# There are almost three times more pneumonia images than normal images, so we will use class weighing

# Define classes
classes = ['NORMAL', 'PNEUMONIA']

# Define class weights
num_pneumonia_train = len(os.listdir(os.path.join(train_dir, classes[1])))
num_normal_train = len(os.listdir(os.path.join(train_dir, classes[0])))
total_train = num_pneumonia_train + num_normal_train
class_weights = torch.tensor([total_train/num_normal_train, total_train/num_pneumonia_train]).to(device)

##3. Preparing Dataset and DataLoader

In [53]:
# Define transforms
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomAffine(degrees=10, translate=(0.05,0.05)),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

val_and_test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

In [54]:
# Define data directories. We won't use a custom class because the dataset is already well-formatted.

train_dataset = ImageFolder('/content/drive/My Drive/chest_xray/train', transform = train_transform)
val_dataset = ImageFolder('/content/drive/My Drive/chest_xray/val', transform = val_and_test_transform)
test_dataset = ImageFolder('/content/drive/My Drive/chest_xray/test', transform = val_and_test_transform)

train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size = 16, shuffle = False)
test_loader = DataLoader(test_dataset, batch_size = 16, shuffle = False)

##4. Defining and Choosing Model

In [55]:
class Bottleneck(nn.Module):

    expansion = 4

    def __init__(self, inplanes, planes, baseWidth, cardinality, stride=1, downsample=None):
      
        # cardinality: num of convolution groups.

        super(Bottleneck, self).__init__()

        D = int(math.floor(planes * (baseWidth / 64)))
        C = cardinality

        self.conv1 = nn.Conv2d(inplanes, D*C, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(D*C)
        self.conv2 = nn.Conv2d(D*C, D*C, kernel_size=3, stride=stride, padding=1, groups=C, bias=False)
        self.bn2 = nn.BatchNorm2d(D*C)
        self.conv3 = nn.Conv2d(D*C, planes * 4, kernel_size=1, stride=1, padding=0, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.relu = nn.ReLU(inplace=True)

        self.downsample = downsample

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNeXt(nn.Module):
    def __init__(self, baseWidth, cardinality, layers, num_classes):

        super(ResNeXt, self).__init__()
        block = Bottleneck

        self.cardinality = cardinality
        self.baseWidth = baseWidth
        self.num_classes = num_classes
        self.inplanes = 64
        self.output_size = 64

        self.conv1 = nn.Conv2d(3, 64, 7, 2, 3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], 2)
        self.layer3 = self._make_layer(block, 256, layers[2], 2)
        self.layer4 = self._make_layer(block, 512, layers[3], 2)
        self.avgpool = nn.AvgPool2d(7)      
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, self.baseWidth, self.cardinality, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, self.baseWidth, self.cardinality))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

# 2 classes
def ResNext50(baseWidth, cardinality):
    model = ResNeXt(baseWidth, cardinality, [3, 4, 6, 3], 2)
    return model


def ResNext101(baseWidth, cardinality):
    model = ResNeXt(baseWidth, cardinality, [3, 4, 23, 3], 2)
    return model


def ResNext152(baseWidth, cardinality):
    model = ResNeXt(baseWidth, cardinality, [3, 8, 36, 3], 2)
    return model

In [56]:
# Initializing model
model = ResNext152(baseWidth = 4, cardinality = 32)

In [57]:
if torch.cuda.is_available():
    model.cuda()

##5. Testing Model

In [58]:
x = torch.randn(8, 3, 224, 224).to(device)
print(model(x).shape) 

torch.Size([8, 2])


##6. Defining Main Training

In [59]:
# Hyper-parameters
num_epochs = 30
learning_rate = 0.001
patience = 10

In [60]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

curr_lr = learning_rate
total_step = len(train_loader)

# Early stopping parameters
early_stopping_counter = 0
best_loss = float('inf')

# For updating learning rate
def update_lr(optimizer, lr):    
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}"
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
    
    # Decay learning rate
    if (epoch+1) % 20 == 0:
        curr_lr /= 3
        update_lr(optimizer, curr_lr)

    # Calculate validation loss
    with torch.no_grad():
        val_loss = 0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            val_loss += criterion(outputs, labels)
        val_loss /= len(val_loader)

        # Save the model if the validation loss is the best observed yet
        if val_loss < best_loss:
            print(f'Saving model with validation loss of {val_loss:.4f}...')
            torch.save(model.state_dict(), 'best_model.pth')
            best_loss = val_loss

    # Early stopping if overfitting
    if early_stopping_counter >= patience:
        print(f'Validation loss has not improved for {patience} epochs. Early stopping...')
        break
    elif val_loss < best_loss:
        best_loss = val_loss
        early_stopping_counter = 0


Epoch [1/30], Step [100/326] Loss: 0.7175
Epoch [1/30], Step [200/326] Loss: 0.7366
Epoch [1/30], Step [300/326] Loss: 0.5713
Saving model with validation loss of 0.6458...
Epoch [2/30], Step [100/326] Loss: 0.5254
Epoch [2/30], Step [200/326] Loss: 0.4545
Epoch [2/30], Step [300/326] Loss: 0.4579
Epoch [3/30], Step [100/326] Loss: 0.6595
Epoch [3/30], Step [200/326] Loss: 0.5817
Epoch [3/30], Step [300/326] Loss: 0.5827
Epoch [4/30], Step [100/326] Loss: 0.5870
Epoch [4/30], Step [200/326] Loss: 0.6499
Epoch [4/30], Step [300/326] Loss: 0.5449
Epoch [5/30], Step [100/326] Loss: 0.4706
Epoch [5/30], Step [200/326] Loss: 0.3053
Epoch [5/30], Step [300/326] Loss: 0.4233
Epoch [6/30], Step [100/326] Loss: 0.4812
Epoch [6/30], Step [200/326] Loss: 0.5639
Epoch [6/30], Step [300/326] Loss: 0.2711
Epoch [7/30], Step [100/326] Loss: 0.6003
Epoch [7/30], Step [200/326] Loss: 0.6206
Epoch [7/30], Step [300/326] Loss: 0.3373
Epoch [8/30], Step [100/326] Loss: 0.5192
Epoch [8/30], Step [200/326] 

##7. Testing

In [61]:
# Load the saved model checkpoint
checkpoint = torch.load('best_model.pth')
model.load_state_dict(checkpoint)

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Model accuracy on test images: {} %'.format(100 * correct / total))

Model accuracy on test images: 92.46794871794872 %
